## Lecture 4 — Data (BCRP + Yahoo) + Plots + Stats + VaR 

Reproduce the key parts of the lecture notebook using:

- **Peru (BCRP API)**: `PD04637PD`, `PD04639PD`, `PD04704XD`, `PD04701XD`  
  *(FX + commodities exactly as in the notebook)*
- **USA (yfinance)**: `SPY`, `TLT`, `GLD`

**Deliverables**
- Multiple **plots** (including **one with annotations**)  
- A **summary statistics table**  
- **Historical 95% VaR** for a **60/40 portfolio** (SPY/TLT)

1. Build (and display) the **BCRPData API URL** that requests the 4 series used in the notebook.  


## Setup

A clean and explicit setup ensures reproducibility: imports are centralised, the sample window is defined once, and display options are set for consistent outputs across machines.


In [1]:
# =========================
# Setup — Lecture 4
# =========================

import numpy as np
import pandas as pd
import requests

# Display options for clear tables in Colab
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

# Sample window (as requested)
START = "2021-01-01"
END   = "2024-12-31"

# BCRP series used in the lecture notebook
BCRP_CODES = ["PD04637PD", "PD04639PD", "PD04704XD", "PD04701XD"]

print("Setup complete.")
print("Sample window:", START, "to", END)
print("BCRP series:", BCRP_CODES)


Setup complete.
Sample window: 2021-01-01 to 2024-12-31
BCRP series: ['PD04637PD', 'PD04639PD', 'PD04704XD', 'PD04701XD']


Constructing and printing the URL makes the data source **auditable** and the request **replicable**.  
The lecture uses the **BCRP Data API** by concatenating series codes with **hyphens (-)**.


In [2]:
# Build the BCRPData API URL
series_str = "-".join(BCRP_CODES)
bcrp_url = f"https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{series_str}/json/{START}/{END}/ing"

print("BCRPData API URL:")
print(bcrp_url)


BCRPData API URL:
https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PD04637PD-PD04639PD-PD04704XD-PD04701XD/json/2021-01-01/2024-12-31/ing


2. Download those series and build a **tidy** table: `date`, `series`, `value`.  


Before reshaping into a wide time-series table, it is best practice to first organise the raw observations into a tidy (long) structure, where each row corresponds to one observation for one series on one date. This format is transparent, facilitates validation by series, and aligns with standard data-science conventions for clean data pipelines.

In [3]:
# --- Download data from the BCRP API ---
response = requests.get(bcrp_url, timeout=60)
response.raise_for_status()
payload = response.json()

# Inspect structure
print("Payload keys:", list(payload.keys()))

# Ensure periods exist and show first observation
assert "periods" in payload and len(payload["periods"]) > 0, "No periods returned. Check URL or date range."

print("Number of periods:", len(payload["periods"]))
print("First period dict:", payload["periods"][0])
print("First 5 raw dates:", [p["name"] for p in payload["periods"][:5]])

Payload keys: ['config', 'periods']
Number of periods: 1043
First period dict: {'name': '04.Jan.21', 'values': ['3.624', '3.624', '359.177118159863', '1943.2']}
First 5 raw dates: ['04.Jan.21', '05.Jan.21', '06.Jan.21', '07.Jan.21', '08.Jan.21']


In [4]:
rows = []
for obs in payload["periods"]:
    date_raw = str(obs["name"])
    vals = obs["values"]

    # Defensive date cleaning:
    # - remove time suffix if present (e.g., "2021-01-04T00:00:00")
    # - strip spaces
    date_clean = date_raw.split("T")[0].strip()

    for code, val in zip(BCRP_CODES, vals):
        rows.append((date_raw, date_clean, code, val))

peru_tidy = pd.DataFrame(rows, columns=["date_raw", "date_clean", "series", "value_raw"])

# Robust date parsing: do NOT force a single format, because your API output may vary
peru_tidy["date"] = pd.to_datetime(peru_tidy["date_clean"], errors="coerce")

# Robust numeric parsing
peru_tidy["value"] = (
    peru_tidy["value_raw"]
    .astype(str)
    .str.replace(",", ".", regex=False)
)
peru_tidy["value"] = pd.to_numeric(peru_tidy["value"], errors="coerce")

# IMPORTANT: check parsing success BEFORE dropping
print("Rows before dropping NaT:", len(peru_tidy))
print("Unparsed dates (NaT):", peru_tidy["date"].isna().sum())

# Now clean and keep the required tidy columns
peru_tidy = (
    peru_tidy
    .dropna(subset=["date"])
    .sort_values(["date", "series"])
    .reset_index(drop=True)
)[["date", "series", "value"]]

# Guaranteed print (works even if display fails)
print("\nTidy table preview (first 12 rows):")
print(peru_tidy.head(12).to_string(index=False))

print("\nTidy table shape:", peru_tidy.shape)
print("Series present:", peru_tidy["series"].unique())


Rows before dropping NaT: 4172
Unparsed dates (NaT): 344

Tidy table preview (first 12 rows):
      date    series       value
2021-01-01 PD04637PD         NaN
2021-01-01 PD04639PD         NaN
2021-01-01 PD04701XD 1891.100000
2021-01-01 PD04704XD  351.148537
2021-01-04 PD04637PD    3.624000
2021-01-04 PD04639PD    3.624000
2021-01-04 PD04701XD 1943.200000
2021-01-04 PD04704XD  359.177118
2021-01-05 PD04637PD    3.627500
2021-01-05 PD04639PD    3.627000
2021-01-05 PD04701XD 1940.350000
2021-01-05 PD04704XD  358.973002

Tidy table shape: (3828, 3)
Series present: ['PD04637PD' 'PD04639PD' 'PD04701XD' 'PD04704XD']


C:\Users\User\AppData\Local\Temp\ipykernel_22588\2888885512.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  peru_tidy["date"] = pd.to_datetime(peru_tidy["date_clean"], errors="coerce")


3. Clean to **wide format** with columns: `fx_interbank`, `fx_sbs`, `gold`, `copper` (as in the notebook).  


A wide-format table is the standard representation for time-series analysis: it stores one observation per date and one column per variable. This structure facilitates direct comparisons across series, straightforward merging with US asset returns, and transparent econometric modelling. The cleaning step consists of mapping raw BCRP codes to semantically meaningful column names and reshaping the tidy dataset via a pivot operation.

In [5]:
# --- Map BCRP codes to the lecture notebook variable names ---
code_to_name = {
    "PD04637PD": "fx_interbank",
    "PD04639PD": "fx_sbs",
    "PD04704XD": "gold",
    "PD04701XD": "copper",
}

# Defensive check: confirm all codes are present before reshaping
missing = set(code_to_name.keys()) - set(peru_tidy["series"].unique())
assert not missing, f"Missing expected series in peru_tidy: {missing}"

# --- Reshape from tidy (long) to wide (time-series) format ---
peru_wide = (
    peru_tidy
    .assign(series=lambda d: d["series"].map(code_to_name))
    .pivot_table(index="date", columns="series", values="value", aggfunc="last")
    .sort_index()
    .reset_index()
)

# Ensure the column order matches the notebook requirement exactly
peru_wide = peru_wide[["date", "fx_interbank", "fx_sbs", "gold", "copper"]]

# Quick inspection
print("Wide table preview:")
print(peru_wide.head(10).to_string(index=False))

print("\nWide table shape:", peru_wide.shape)
print("Columns:", list(peru_wide.columns))

Wide table preview:
      date  fx_interbank  fx_sbs       gold  copper
2021-01-01           NaN     NaN 351.148537 1891.10
2021-01-04      3.624000   3.624 359.177118 1943.20
2021-01-05      3.627500   3.627 358.973002 1940.35
2021-01-06      3.624833   3.625 368.747917 1931.95
2021-01-07      3.620667   3.620 364.529508 1920.10
2021-01-08      3.611000   3.610 369.496345 1862.90
2021-01-11      3.615167   3.615 360.673973 1847.25
2021-01-12      3.606333   3.606 362.102789 1841.25
2021-01-13      3.612500   3.610 361.082206 1858.85
2021-01-14      3.610667   3.610 362.987294 1841.75

Wide table shape: (957, 5)
Columns: ['date', 'fx_interbank', 'fx_sbs', 'gold', 'copper']


4. Download `SPY`, `TLT`, `GLD` from yfinance and build: `date`, `ticker`, `close`.  


US financial assets are retrieved using Yahoo Finance via the yfinance package, which is widely adopted in empirical finance for obtaining daily market data. Adjusted prices are used to ensure that the resulting series account for dividends and stock splits, thereby making subsequent return calculations economically meaningful. The data are organised in a tidy (long) format, with one observation per asset and date, which is consistent with best practices for financial time-series analysis.

In [6]:
import yfinance as yf
import pandas as pd

USA_TICKERS = ["SPY", "TLT", "GLD"]

yf_data = yf.download(
    tickers=USA_TICKERS,
    start=START,
    end=END,
    auto_adjust=True,
    progress=False
)

print("Empty?", yf_data.empty)
print("Columns names:", getattr(yf_data.columns, "names", None))
print("Columns sample:", list(yf_data.columns)[:5])

# --- Handle MultiIndex with order (Price, Ticker) ---
if isinstance(yf_data.columns, pd.MultiIndex):
    # Your case: level 0 is Price field, level 1 is Ticker
    lvl0 = yf_data.columns.get_level_values(0)
    lvl1 = yf_data.columns.get_level_values(1)

    if "Close" in set(lvl0):
        close_wide = yf_data["Close"].copy()     # columns become tickers
    elif "close" in set(lvl0):
        close_wide = yf_data["close"].copy()
    else:
        raise KeyError(f"No Close field in level 0. Found level0 fields: {sorted(set(lvl0))}")

    usa_tidy = (
        close_wide
        .reset_index()
        .melt(id_vars="Date", var_name="ticker", value_name="close")
        .rename(columns={"Date": "date"})
    )

else:
    # Single-index fallback
    if "Close" in yf_data.columns:
        usa_tidy = yf_data[["Close"]].rename(columns={"Close": "close"}).reset_index()
    elif "close" in yf_data.columns:
        usa_tidy = yf_data[["close"]].rename(columns={"close": "close"}).reset_index()
    else:
        raise KeyError(f"No Close column found. Columns: {list(yf_data.columns)}")

    usa_tidy = usa_tidy.rename(columns={"Date": "date"})
    usa_tidy["ticker"] = USA_TICKERS[0]
    usa_tidy = usa_tidy[["date", "ticker", "close"]]

# Clean types and order
usa_tidy["date"] = pd.to_datetime(usa_tidy["date"])
usa_tidy["close"] = pd.to_numeric(usa_tidy["close"], errors="coerce")
usa_tidy = (
    usa_tidy
    .dropna(subset=["close"])
    .sort_values(["date", "ticker"])
    .reset_index(drop=True)
)[["date", "ticker", "close"]]

print("\nUS assets (tidy) — preview:")
print(usa_tidy.head(12).to_string(index=False))

print("\nTidy shape:", usa_tidy.shape)
print("Tickers present:", usa_tidy["ticker"].unique())

Empty? False
Columns names: ['Price', 'Ticker']
Columns sample: [('Close', 'GLD'), ('Close', 'SPY'), ('Close', 'TLT'), ('High', 'GLD'), ('High', 'SPY')]

US assets (tidy) — preview:
      date ticker      close
2021-01-04    GLD 182.330002
2021-01-04    SPY 344.256714
2021-01-04    TLT 134.713715
2021-01-05    GLD 182.869995
2021-01-05    SPY 346.627716
2021-01-05    TLT 133.713226
2021-01-06    GLD 179.899994
2021-01-06    SPY 348.700073
2021-01-06    TLT 130.968384
2021-01-07    GLD 179.479996
2021-01-07    SPY 353.880859
2021-01-07    TLT 129.813950

Tidy shape: (3012, 3)
Tickers present: ['GLD' 'SPY' 'TLT']


5. Compute **daily returns** by ticker (`ret`) and validate there are **no inf values**.  


Daily returns are computed as percentage changes in adjusted closing prices, which is the standard transformation in empirical finance for analysing asset performance and risk. Returns are calculated by ticker to ensure that observations are not incorrectly mixed across assets. After computing returns, it is essential to validate that no infinite values are present, as these may arise from zero prices or data glitches and would invalidate subsequent statistical analysis.

In [7]:
# --- Compute daily returns by ticker ---
usa_returns = usa_tidy.copy()

usa_returns["ret"] = (
    usa_returns
    .groupby("ticker")["close"]
    .pct_change()
)

# --- Validate: check for infinite values ---
has_inf = np.isinf(usa_returns["ret"]).any()
print("Any infinite return values?", has_inf)

# Replace infinite values with NaN (defensive cleaning)
usa_returns.loc[np.isinf(usa_returns["ret"]), "ret"] = np.nan

# Final check
assert not np.isinf(usa_returns["ret"]).any(), "Infinite values still present in returns!"

# Preview
print("\nUS assets with returns — preview:")
print(
    usa_returns
    .head(12)
    .to_string(index=False)
)

Any infinite return values? False

US assets with returns — preview:
      date ticker      close       ret
2021-01-04    GLD 182.330002       NaN
2021-01-04    SPY 344.256714       NaN
2021-01-04    TLT 134.713715       NaN
2021-01-05    GLD 182.869995  0.002962
2021-01-05    SPY 346.627716  0.006887
2021-01-05    TLT 133.713226 -0.007427
2021-01-06    GLD 179.899994 -0.016241
2021-01-06    SPY 348.700073  0.005979
2021-01-06    TLT 130.968384 -0.020528
2021-01-07    GLD 179.479996 -0.002335
2021-01-07    SPY 353.880859  0.014857
2021-01-07    TLT 129.813950 -0.008815


6. *(Quantities)* Compare FX levels in Peru: produce a **plot** and a short comment.  


7. *(Proportions)* Compute the **share of positive-return days** by ticker (USA).  


8. Plot that share as a **bar chart** and add **labels above each bar** (`annotate`).  


9. *(Distributions)* Compare the distribution of **Peru Gold** vs **GLD** (histogram).  


10. Add an **ECDF** (if used in the notebook) and comment on what changes vs the histogram.  


11. *(Relationships)* Build `FX_change` and relate it to `SPY_ret` (scatter plot).  


12. Compute the **correlation** between `FX_change` and `SPY_ret` and explain the sign.  


13. Estimate a simple regression `FX_change ~ SPY_ret` and interpret the coefficient.  


14. *(Pandas)* Do a selection exercise: `.iloc` (position-based) vs conditional filtering.  


15. Create missing data on purpose in one series and apply imputation (as in the notebook).  


16. Standardize a variable (z-score) and plot **before vs after**.  


17. Find the day with the largest `|SPY_ret|` and **annotate it** in the returns plot (like the exercise).  


18. Save one figure into `/figures` using `savefig` and verify the file exists.  


19. Build a **summary stats table** for returns (mean, sd, p5, p95, etc.).  


20. Compute **historical 95% VaR** for a **60/40 portfolio (SPY/TLT)** and explain what it means.
